# Importaciones

In [1]:
import pandas as pd
import numpy as np
# from sklearn.preprocessing import OrdinalEncoder

import os
import sys

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

import src.analysis as an

c:\Users\Laura\anaconda3\envs\ox_eafit\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Lectura de los datos

In [2]:
# Carga el archivo xlsx
medellin_csv = "../data/output/table/encuestas/Encuesta caminata (Medellín)(1-613).xlsx"

# Lee el archivo CSV en un DataFrame
df_medellin = pd.read_excel(medellin_csv)

In [3]:
# Carga el archivo CSV
guadalajara_csv = "../data/output/table/encuestas/Encuesta caminata (Guadalajara)(1-553).xlsx"

# Lee el archivo CSV en un DataFrame
df_guadalajara = pd.read_excel(guadalajara_csv)

## Eliminación de encuestas de Valentina Vasquez

In [4]:
print(len(df_medellin))
df_medellin = df_medellin[df_medellin['Encuestador']!='Valentina_Vasquez']
print(len(df_medellin))

613
535


## Eliminación de columnas que no son de nuestro interés
No se tienen en cuenta personas con discapacidad, y personas que no hacen uso de ningún equipamiento

In [5]:
df_medellin = df_medellin[df_medellin['¿Hace uso de algún equipamiento/sitio de interés localizado en el sector/territorio?'] == 'Si']
df_guadalajara = df_guadalajara[df_guadalajara['¿Hace uso de algún equipamiento/sitio de interés localizado en el sector/territorio?'] == 'Si']

df_medellin = df_medellin[df_medellin['¿Se encuentra usted en alguna condición de discapacidad?'] != 'Si']
df_guadalajara = df_guadalajara[df_guadalajara['¿Se encuentra usted en alguna condición de discapacidad?'] != 'Si']
# df_medellin = df_medellin.drop(columns=['ID', 'Hora de inicio', 'Correo electrónico', 'Nombre', 'Hora de la última modificación', 
df_medellin = df_medellin.drop(columns=['Hora de inicio', 'Correo electrónico', 'Nombre', 'Hora de la última modificación',
                    'Fecha de realización', 'Encuestador', 'Encuesta #', 'Hora de finalización', 
                    'Lugar donde se realizó la encuesta',
                    'Lugar donde se realizó la encuesta (Moravia)', 'Lugar donde se realizó la encuesta (Floresta)', 
                    'Lugar donde se realizó la encuesta (Aguacatala)',
                    '¿Hace uso de algún equipamiento/sitio de interés localizado en el sector/territorio?', ##
                    '¿Cómo suple las necesidades de equipamientos educativos?', ##
                    '¿Cómo suple las necesidades de equipamientos de cultura o deportivos?', ##
                    '¿Se encuentra usted en alguna condición de discapacidad?', ###
                    'Indique cuál de las siguientes condiciones de discapacidad presenta', ###
                    'Indique el lugar donde inició de su recorrido lo más preciso posible (Punto de referencia, intersección más cercana)', 
                    'Indique la hora de salida de su lugar de origen de recorrido (formato militar)', 'Ingrese los minutos de salida', 
                    'Indique el lugar de destino de su recorrido lo más precios posible (Punto de referencia, intersección más cercana) (old)',
                    'Indique el lugar de destino de su recorrido',
                    '¿Cuánto tiempo le tomó el recorrido caminando para legar a su lugar de destino? (Ingresar en minutos)', 
                    'Describa la ruta que tomó para este último viaje desde su lugar de origen hasta su lugar de destino. Puede utilizar direcciones y/o lugares conocidos de referencia para facilitar la identificación...'], ###
                               axis=1)
df_guadalajara = df_guadalajara.drop(columns=['ID', 'Hora de inicio', 'Correo electrónico', 'Nombre', 'Hora de la última modificación', 
                    'Fecha de realización', 'Encuestador', 'Encuesta #', 'Hora de finalización',
                    '¿Hace uso de algún equipamiento/sitio de interés localizado en el sector/territorio?', ##
                    '¿Cómo suple las necesidades de equipamientos educativos?', ##
                    '¿Cómo suple las necesidades de equipamientos de cultura o deportivos?', ##
                    'Estrato socioeconómico (NO APLICA)', # Está lleno de NaN
                    '¿Se encuentra usted en alguna condición de discapacidad?', ###
                    'Indique cuál de las siguientes condiciones de discapacidad presenta', ###
                    'Indique el lugar donde inició de su recorrido lo más preciso posible (Punto de referencia, intersección más cercana)', 
                    'Indique la hora de salida de su lugar de origen de recorrido (formato militar)', 'Ingrese los minutos de salida', 
                    'Indique el lugar de destino de su recorrido lo más precios posible (Punto de referencia, intersección más cercana)', 
                    '¿Cuánto tiempo le tomó el recorrido caminando para legar a su lugar de destino? (Ingresar en minutos)', 
                    'Describa la ruta que tomó para este último viaje desde su lugar de origen hasta su lugar de destino. Puede utilizar direcciones y/o lugares conocidos de referencia para facilitar la identificación...'], ###
                               axis=1)

In [6]:
print(df_medellin.shape)
print(df_guadalajara.shape)

(502, 45)
(462, 44)


# Estandarización de formatos

In [7]:
# Para dataframe de guadalajara
nombres_nuevos = {
    'Estrato socioeconómico (NO APLICA)': 'Estrato socioeconómico',
    'La cercanía del equipamiento X': 'Qué tan importante es para usted la cercanía del equipamiento X',
    'La presencia de andenes': 'Qué tan importante es para usted la presencia de andenes',
    'Sentirme seguro/a frente a otros vehículos': 'Qué tan importante es para usted sentirse seguro/a frente a otros vehículos',
    'Sentirme seguro/a frente a posibles delitos en mi trayecto': 'Qué tan importante es para usted sentirse seguro/a frente a posibles delitos en su trayecto',
    'La posibilidad de hacer múltiples vueltas/trámites en mi recorrido': 'Qué tan importante es para usted la posibilidad de hacer múltiples vueltas/trámites en su recorrido',
    'La posibilidad de poder tomar desvíos y hacer múltiples trayectos en mi recorrido': 'Qué tan importante es para usted la posibilidad de poder tomar desvíos y hacer múltiples trayectos en su recorrido',
    'Sentirme cómodo/a por la sombra generada por la vegetación': 'Qué tan importante es para usted sentirse cómodo/a por la sombra generada por la vegetación',
    'Sentirme cómodo/a en términos de la cantidad de gente a lo largo de mi recorrido': 'Qué tan importante es para usted sentirse cómodo/a en términos de la cantidad de gente a lo largo de mi recorrido',
    'Que el paisaje sea agradable': 'Qué tan importante es para usted que el paisaje sea agradable',
    'Column': '36. A lo largo de su recorrido, ¿Sientes que hay personas a tu alrededor que pudieran apoyarte en caso de algún problema?',
    '2': '37. A lo largo de su recorrido, ¿Sientes que hay diversas intersecciones que te permiten cruzar las calles de manera segura?',
    '3': '38. ¿Se te hace largo y aburrido el recorrido?',
    '4': '39. A lo largo de tu recorrido, ¿Es posible desviarse en múltiples puntos/intersecciones para cambiar de ruta?',
    '5': '40. A lo largo de tu recorrido, ¿Puedes aprovechar para acceder a múltiples servicios y satisfacer necesidades diversas (realizar vueltas/pendientes, comprar alimentos, divertirse, etc.)?',
    '6': '41. A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?',
    '7': '42. A lo largo de tu recorrido, ¿Encuentras elementos que hacen que tu caminata sea poco monótona?',
    '8': '43. A lo largo de tu recorrido, ¿Sientes que el tipo de vegetación te genera sombra y protección del sol?',
    '9': '44. A lo largo de tu recorrido, ¿Sientes que la presencia y el tipo de vegetación crea un entorno agradable?',
    '10': '45. A lo largo de tu recorrido, ¿Consideras que el hecho de caminar te implicó un gran esfuerzo físico debido a la pendiente/inclinación?',
    '11': '46. A lo largo de tu recorrido, ¿Te sientes seguro(a) ante la presencia de vehículos?',
    '12': '47. A lo largo de tu recorrido, ¿Sientes que la continuidad de andenes te permite caminar de manera cómoda (no hay: cambios de niveles, interrupciones repentinas del andén, presencia de obstáculos, cambios en el ancho del andén, etc.)?',
    '¿Cuáles?': '¿Cuáles equipamientos de interés en el sector utilizas?'
}

# Renombrar las columnas específicas  
df_guadalajara.rename(columns=nombres_nuevos, inplace=True)
df_medellin.rename(columns=nombres_nuevos, inplace=True)

# División por caminantes y no caminantes

In [8]:
med_caminantes = df_medellin[df_medellin["¿Qué medio de transporte utilizó para llegar al equipamiento X? (último viaje realizado)"] == "Caminata"]
med_no_caminantes = df_medellin[df_medellin["¿Qué medio de transporte utilizó para llegar al equipamiento X? (último viaje realizado)"] != "Caminata"]

gua_caminantes = df_guadalajara[df_guadalajara["¿Qué medio de transporte utilizó para llegar al equipamiento X? (último viaje realizado)"] == "Caminata"]
gua_no_caminantes = df_guadalajara[df_guadalajara["¿Qué medio de transporte utilizó para llegar al equipamiento X? (último viaje realizado)"] != "Caminata"]

In [9]:
print(df_medellin.shape)
print(med_caminantes.shape)
print(med_no_caminantes.shape)

(502, 45)
(403, 45)
(99, 45)


Eliminación de columnas no acordes con los caminantes

In [10]:
# Se elimina campo de transporte porque ya se filtró,
# y se eliminan columnas asociadas a personas que usan otro medio de transporte
med_caminantes = med_caminantes.drop(columns=['¿Qué medio de transporte utilizó para llegar al equipamiento X? (último viaje realizado)',
                                            'En una semana típica, ¿Cuántas veces realiza este mismo viaje utilizando el mismo modo de transporte?',
                                            'Debido a la lejanía del equipamiento X', 'Debido a la ausencia de andenes',
                                            'Porque no me siento seguro/a frente a otros vehículos', 
                                            'Porque no me siento seguro/a frente a posibles delitos en mi trayecto', 
                                            'Debido al esfuerzo que debo hacer a causa de la pendiente de las calles', 
                                            'Debido a que no me siento cómodo/a por el calor', 'Debido a que siento que los andenes muy saturados de gente', 
                                            'Debido a que el paisaje a lo largo del recorrido no es agradable'], axis=1)
# Se elimina campo de transporte porque ya se filtró,
# y se eliminan columnas asociadas a personas que usan otro medio de transporte
gua_caminantes = gua_caminantes.drop(columns=['¿Qué medio de transporte utilizó para llegar al equipamiento X? (último viaje realizado)',
                                            'En una semana típica, ¿Cuántas veces realiza este mismo viaje utilizando el mismo modo de transporte?',
                                            'Debido a la lejanía del equipamiento X', 'Debido a la ausencia de andenes',
                                            'Porque no me siento seguro/a frente a otros vehículos', 
                                            'Porque no me siento seguro/a frente a posibles delitos en mi trayecto', 
                                            'Debido al esfuerzo que debo hacer a causa de la pendiente de las calles', 
                                            'Debido a que no me siento cómodo/a por el calor', 'Debido a que siento que los andenes muy saturados de gente', 
                                            'Debido a que el paisaje a lo largo del recorrido no es agradable'], axis=1)

In [11]:
med_caminantes.dropna(axis=0, inplace=True)
gua_caminantes.dropna(axis=0, inplace=True)
print(med_caminantes.shape)
print(gua_caminantes.shape)

(403, 35)
(178, 34)


In [12]:
print(med_caminantes.isnull().sum().sum())
print(gua_caminantes.isnull().sum().sum())

0
0


# Corrección de Formatos
Convertir a numérico

In [13]:
lista_columnas = ["Género", "Edad (años)", "Estrato socioeconómico", "Nivel educativo completado", "En una semana típica, ¿Cuántas veces realiza este mismo viaje utilizando la caminata como modo de transporte? (CAMINATA)"]
orden = [['Femenino','Masculino'], 
         ['10-19', '20-29', '30-39', '40-49', '50-59', 'Mayor a 60'], 
         ['E1', 'E2', 'E3', 'E4', 'E5', 'E6'], 
         ['Ninguno', 'Primaria', 'Secundaria', 'Técnico', 'Tecnológico', 'Pregrado', 'Posgrado'], 
         ['Muy poco frecuente', '1', '2', '3', '4', '5 o más']]

for i, columna in enumerate(lista_columnas):
    med_caminantes = an.ordinal_encoding(med_caminantes, columna, orden[i])
    if columna != "Estrato socioeconómico":
        gua_caminantes = an.ordinal_encoding(gua_caminantes, columna, orden[i])

In [14]:
med_caminantes = med_caminantes.rename(columns={'Edad (años)': 'Rango edad (10 años)'})
med_caminantes = med_caminantes.rename(columns={'¿Cuáles?': '¿Cuáles equipamientos de interés en el sector utilizas?'})

gua_caminantes = gua_caminantes.rename(columns={'Edad (años)': 'Rango edad (10 años)'})
gua_caminantes = gua_caminantes.rename(columns={'¿Cuáles?': '¿Cuáles equipamientos de interés en el sector utilizas?'})

## Corrección de preguntas de importancia y percepción
Trasladar a rango (0,1)

In [15]:
# Pasar de datos cualitativos a cuantitativos en el rango [0,1]
columnas_importancia = ['Qué tan importante es para usted la cercanía del equipamiento X', 
                       'Qué tan importante es para usted la presencia de andenes', 
                       'Qué tan importante es para usted sentirse seguro/a frente a otros vehículos', 
                       'Qué tan importante es para usted sentirse seguro/a frente a posibles delitos en su trayecto', 
                       'Qué tan importante es para usted la posibilidad de hacer múltiples vueltas/trámites en su recorrido', 
                       'Qué tan importante es para usted la posibilidad de poder tomar desvíos y hacer múltiples trayectos en su recorrido', 
                       'Qué tan importante es para usted sentirse cómodo/a por la sombra generada por la vegetación', 
                       'Qué tan importante es para usted sentirse cómodo/a en términos de la cantidad de gente a lo largo de mi recorrido', 
                       'Qué tan importante es para usted que el paisaje sea agradable']
# Defino la jerarquía u orden de las categorías, de menor a mayor
# (el número que se le asignará a cada uno dependerá de esto: 0 al primer elemento, 1 al segundo, etc...)
orden = ['No es importante', 'Es poco importante', 'Moderadamente importante', 'Importante',   'Muy importante']
    
for columna in columnas_importancia:
    med_caminantes = an.ordinal_encoding(med_caminantes, columna, orden)
    med_caminantes[columna] = med_caminantes[columna]*0.25
    med_caminantes[columna] = pd.to_numeric(med_caminantes[columna])
    
    gua_caminantes = an.ordinal_encoding(gua_caminantes, columna, orden)
    gua_caminantes[columna] = gua_caminantes[columna]*0.25
    gua_caminantes[columna] = pd.to_numeric(gua_caminantes[columna])

In [16]:
# Pasar de datos cualitativos a cuantitativos en el rango [0,1]
columnas_percepcion = ['36. A lo largo de su recorrido, ¿Sientes que hay personas a tu alrededor que pudieran apoyarte en caso de algún problema?', 
                       '37. A lo largo de su recorrido, ¿Sientes que hay diversas intersecciones que te permiten cruzar las calles de manera segura?', 
                       '38. ¿Se te hace largo y aburrido el recorrido?', 
                       '39. A lo largo de tu recorrido, ¿Es posible desviarse en múltiples puntos/intersecciones para cambiar de ruta?', 
                       '40. A lo largo de tu recorrido, ¿Puedes aprovechar para acceder a múltiples servicios y satisfacer necesidades diversas (realizar vueltas/pendientes, comprar alimentos, divertirse, etc.)?', 
                       '41. A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?', 
                       '42. A lo largo de tu recorrido, ¿Encuentras elementos que hacen que tu caminata sea poco monótona?', 
                       '43. A lo largo de tu recorrido, ¿Sientes que el tipo de vegetación te genera sombra y protección del sol?', 
                       '44. A lo largo de tu recorrido, ¿Sientes que la presencia y el tipo de vegetación crea un entorno agradable?', 
                       '45. A lo largo de tu recorrido, ¿Consideras que el hecho de caminar te implicó un gran esfuerzo físico debido a la pendiente/inclinación?', 
                       '46. A lo largo de tu recorrido, ¿Te sientes seguro(a) ante la presencia de vehículos?', 
                       '47. A lo largo de tu recorrido, ¿Sientes que la continuidad de andenes te permite caminar de manera cómoda (no hay: cambios de niveles, interrupciones repentinas del andén, presencia de obstáculos, cambios en el ancho del andén, etc.)?']

# Defino la jerarquía u orden de las categorías, de menor a mayor
# (el número que se le asignará a cada uno dependerá de esto: 0 al primer elemento, 1 al segundo, etc...)
orden = ['Totalmente en desacuerdo', 'En desacuerdo', 'Ni de acuerdo, ni en desacuerdo', 'De acuerdo', 'Totalmente de acuerdo']
    
for columna in columnas_percepcion:
    med_caminantes = an.ordinal_encoding(med_caminantes, columna, orden)
    med_caminantes[columna] = med_caminantes[columna]*0.25
    med_caminantes[columna] = pd.to_numeric(med_caminantes[columna])
    
    gua_caminantes = an.ordinal_encoding(gua_caminantes, columna, orden)
    gua_caminantes[columna] = gua_caminantes[columna]*0.25
    gua_caminantes[columna] = pd.to_numeric(gua_caminantes[columna])

Corregir sentido de preguntas, para que todas aporten al índice de forma positiva

In [17]:
med_caminantes['38. ¿Se te hace largo y aburrido el recorrido?'] = 1 - med_caminantes['38. ¿Se te hace largo y aburrido el recorrido?']
med_caminantes = med_caminantes.rename(columns={'38. ¿Se te hace largo y aburrido el recorrido?': '38. ¿Se te hace corto y agradable el recorrido?'})
med_caminantes['41. A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?'] = 1 - med_caminantes['41. A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?']
med_caminantes = med_caminantes.rename(columns={'41. A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?': '41. A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar NO se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?'})
med_caminantes['45. A lo largo de tu recorrido, ¿Consideras que el hecho de caminar te implicó un gran esfuerzo físico debido a la pendiente/inclinación?'] = 1 - med_caminantes['45. A lo largo de tu recorrido, ¿Consideras que el hecho de caminar te implicó un gran esfuerzo físico debido a la pendiente/inclinación?']
med_caminantes = med_caminantes.rename(columns={'45. A lo largo de tu recorrido, ¿Consideras que el hecho de caminar te implicó un gran esfuerzo físico debido a la pendiente/inclinación?': '45. A lo largo de tu recorrido, ¿Consideras que el hecho de caminar NO te implicó un gran esfuerzo físico debido a la pendiente/inclinación?'})

gua_caminantes['38. ¿Se te hace largo y aburrido el recorrido?'] = 1 - gua_caminantes['38. ¿Se te hace largo y aburrido el recorrido?']
gua_caminantes = gua_caminantes.rename(columns={'38. ¿Se te hace largo y aburrido el recorrido?': '38. ¿Se te hace corto y agradable el recorrido?'})
gua_caminantes['41. A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?'] = 1 - gua_caminantes['41. A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?']
gua_caminantes = gua_caminantes.rename(columns={'41. A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?': '41. A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar NO se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?'})
gua_caminantes['45. A lo largo de tu recorrido, ¿Consideras que el hecho de caminar te implicó un gran esfuerzo físico debido a la pendiente/inclinación?'] = 1 - gua_caminantes['45. A lo largo de tu recorrido, ¿Consideras que el hecho de caminar te implicó un gran esfuerzo físico debido a la pendiente/inclinación?']
gua_caminantes = gua_caminantes.rename(columns={'45. A lo largo de tu recorrido, ¿Consideras que el hecho de caminar te implicó un gran esfuerzo físico debido a la pendiente/inclinación?': '45. A lo largo de tu recorrido, ¿Consideras que el hecho de caminar NO te implicó un gran esfuerzo físico debido a la pendiente/inclinación?'})

Guardar nombre de columnas en listas

In [18]:
columnas_importancia = ['Qué tan importante es para usted sentirse seguro/a frente a posibles delitos en su trayecto',

                        'Qué tan importante es para usted la posibilidad de poder tomar desvíos y hacer múltiples trayectos en su recorrido', 
 
                        'Qué tan importante es para usted la posibilidad de hacer múltiples vueltas/trámites en su recorrido',
                        'Qué tan importante es para usted sentirse cómodo/a en términos de la cantidad de gente a lo largo de mi recorrido',
                        
                        'Qué tan importante es para usted sentirse cómodo/a por la sombra generada por la vegetación', 
                        'Qué tan importante es para usted que el paisaje sea agradable',    
                        
                        'Qué tan importante es para usted sentirse seguro/a frente a otros vehículos', 
                        'Qué tan importante es para usted la presencia de andenes',
                        
                        'Qué tan importante es para usted la cercanía del equipamiento X']

columnas_percepcion = ['36. A lo largo de su recorrido, ¿Sientes que hay personas a tu alrededor que pudieran apoyarte en caso de algún problema?', 
                       
                       '37. A lo largo de su recorrido, ¿Sientes que hay diversas intersecciones que te permiten cruzar las calles de manera segura?', 
                       '38. ¿Se te hace corto y agradable el recorrido?', 
                       '39. A lo largo de tu recorrido, ¿Es posible desviarse en múltiples puntos/intersecciones para cambiar de ruta?', 
                       
                       '40. A lo largo de tu recorrido, ¿Puedes aprovechar para acceder a múltiples servicios y satisfacer necesidades diversas (realizar vueltas/pendientes, comprar alimentos, divertirse, etc.)?', 
                       '41. A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar NO se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?', 
                       '42. A lo largo de tu recorrido, ¿Encuentras elementos que hacen que tu caminata sea poco monótona?', 
                       
                       '43. A lo largo de tu recorrido, ¿Sientes que el tipo de vegetación te genera sombra y protección del sol?', 
                       '44. A lo largo de tu recorrido, ¿Sientes que la presencia y el tipo de vegetación crea un entorno agradable?', 
                       
                       '45. A lo largo de tu recorrido, ¿Consideras que el hecho de caminar NO te implicó un gran esfuerzo físico debido a la pendiente/inclinación?', 
                       
                       '46. A lo largo de tu recorrido, ¿Te sientes seguro(a) ante la presencia de vehículos?', 
                       '47. A lo largo de tu recorrido, ¿Sientes que la continuidad de andenes te permite caminar de manera cómoda (no hay: cambios de niveles, interrupciones repentinas del andén, presencia de obstáculos, cambios en el ancho del andén, etc.)?']

# Betas variables demográficas

In [19]:
TAs = [med_caminantes,gua_caminantes]
variables_demograficas = {'Género':['Femenino','Masculino'],
                          'Rango edad (10 años)':['10-19', '20-29', '30-39', '40-49', '50-59', 'Mayor a 60'],
                          'Estrato socioeconómico':['E1', 'E2', 'E3', 'E4', 'E5', 'E6']}

# TA = TAs[0]
for TA in TAs:
    if len(TA) == len(med_caminantes):
        city = 'Medellin'
    else:
        city = 'Guadalajara'

    if city == 'Guadalajara':
        del variables_demograficas['Estrato socioeconómico']

    for key in variables_demograficas.keys():
        for number in range(len(variables_demograficas[key])):
            # Segmentacion de los tipos de preguntas
            percepcion_genero = TA[TA[key] == number][columnas_percepcion]
            importancia_genero = TA[TA[key] == number][columnas_importancia]

            # Segmentacion de las preguntas de percepcion
            per_densidad_pob = percepcion_genero[columnas_percepcion[0]]
            per_densidad_inter = percepcion_genero[columnas_percepcion[1:4]]
            per_usos_suelo = percepcion_genero[columnas_percepcion[4:7]]
            per_ind_vegetacion = percepcion_genero[columnas_percepcion[7:9]]
            per_pendiente = percepcion_genero[columnas_percepcion[9]]
            per_presenc_aceras = percepcion_genero[columnas_percepcion[10:]]

            # Calculo de los betas por variables fisicas por genero
            per_campos = [per_densidad_pob, per_densidad_inter, per_usos_suelo, per_ind_vegetacion, per_pendiente, per_presenc_aceras]
            lista = ['densidad_pob','densidad_inter','usos_suelo','ind_vegetacion','pendiente','presenc_aceras']
            per_betas_campos = {}
            for i in range(len(per_campos)):
                per_betas_campos[lista[i]] = per_campos[i].mean().mean()

            df_betas = pd.DataFrame(list(per_betas_campos.items()), columns=['variable', 'beta']).round(4)
            change = {'densidad_pob':'den_pop','densidad_inter':'den_inter','usos_suelo':'land_use',
                    'ind_vegetacion':'ind_ndvi','pendiente':'grade_abs','presenc_aceras':'sidewalk'}
            df_betas = df_betas.replace({'variable':change})
            print(city)
            print(key,':', number, 'perception')
            print(df_betas)

            # Segmentacion de las preguntas de importancia
            imp_densidad_pob = importancia_genero[columnas_importancia[0]]
            imp_densidad_inter = importancia_genero[columnas_importancia[1]]
            imp_usos_suelo = importancia_genero[columnas_importancia[2:4]]
            imp_ind_vegetacion = importancia_genero[columnas_importancia[4:6]]
            imp_cercania = importancia_genero[columnas_importancia[8]]
            imp_presenc_aceras = importancia_genero[columnas_importancia[6:8]]

            # Calculo de los betas por variables fisicas por genero
            imp_campos = [imp_densidad_pob, imp_densidad_inter, imp_usos_suelo, imp_ind_vegetacion, imp_presenc_aceras]
            lista_imp = ['densidad_pob', 'densidad_inter', 'usos_suelo', 'ind_vegetacion', 'presenc_aceras']
            imp_betas_campos = {}
            for i in range(len(imp_campos)):
                imp_betas_campos[lista_imp[i]] = imp_campos[i].mean().mean()

            df_betas_imp = pd.DataFrame(list(imp_betas_campos.items()), columns=['variable', 'beta']).round(4)
            change = {'densidad_pob':'den_pop','densidad_inter':'den_inter','usos_suelo':'land_use',
                    'ind_vegetacion':'ind_ndvi','presenc_aceras':'sidewalk'}
            df_betas_imp = df_betas_imp.replace({'variable':change})
            print(city)
            print(key,':', number,'importance')
            print(df_betas_imp)

Medellin
Género : 0 perception
    variable    beta
0    den_pop  0.6618
1  den_inter  0.6754
2   land_use  0.5711
3   ind_ndvi  0.6592
4  grade_abs  0.7224
5   sidewalk  0.6138
Medellin
Género : 0 importance
    variable    beta
0    den_pop  0.7250
1  den_inter  0.6158
2   land_use  0.6836
3   ind_ndvi  0.7553
4   sidewalk  0.7868
Medellin
Género : 1 perception
    variable    beta
0    den_pop  0.6573
1  den_inter  0.6995
2   land_use  0.5736
3   ind_ndvi  0.6784
4  grade_abs  0.7453
5   sidewalk  0.6203
Medellin
Género : 1 importance
    variable    beta
0    den_pop  0.7089
1  den_inter  0.5646
2   land_use  0.6021
3   ind_ndvi  0.6984
4   sidewalk  0.7306
Medellin
Rango edad (10 años) : 0 perception
    variable    beta
0    den_pop  0.6533
1  den_inter  0.6643
2   land_use  0.5768
3   ind_ndvi  0.6600
4  grade_abs  0.7130
5   sidewalk  0.6060
Medellin
Rango edad (10 años) : 0 importance
    variable    beta
0    den_pop  0.6440
1  den_inter  0.5864
2   land_use  0.6240
3   ind_n